In [ ]:
print("hi")

In [ ]:
# Set logging for the queries
import logging
import pickle
from langchain.vectorstores.base import VectorStoreRetriever
logging.basicConfig()
logging.getLogger("langchain.retrievers.mlti_query").setLevel(logging.INFO)

In [ ]:
to_file = "vector_src/ss_src-vectorstore.pkl"
with open(to_file, "rb") as f:
    vectorstore = pickle.load(f)
retriever = VectorStoreRetriever(vectorstore=vectorstore)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI
from query_data import get_llm

llm=get_llm('local')
retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

In [ ]:
llm.predict("你是谁?你和gpt-3.5的差距有多大")

In [ ]:
question = "国家的税收策略是什么?"

docs = retriever.get_relevant_documents(query=question)
len(docs)

In [ ]:
docs

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

QA_PROMPT = PromptTemplate(
    input_variables=["query", "contexts"],
    template="""You are a helpful assistant who answers user queries using the
    contexts provided. If the question cannot be answered using the information
    provided say "I don't know,必须使用中文输出!".
    Contexts:
    {contexts}

    Question: {query}""",
)

# Chain
qa_chain = LLMChain(llm=llm, prompt=QA_PROMPT)

In [ ]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import SequentialChain
from langchain.chains import TransformChain

# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

template = """
Your task is to generate 5 different search queries that aim to
answer the user question from multiple perspectives. 
Each query MUST tackle the question from a different viewpoint, we
want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.You answer MUST in Chinese output.
Original question: {question}
"""

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template=template,
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

In [ ]:
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
docs = retriever.get_relevant_documents(
    query=question
)
len(docs)

In [ ]:
from langchain.chains import TransformChain

def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs)
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

In [ ]:
retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

rag_chain = SequentialChain(
    chains=[retrieval_chain, qa_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text"]
)

In [ ]:
out = rag_chain({"question": question})
out["text"]

In [1]:
class MyExampleClass:
    def __init__(self, value):
        self.value = value

    def display(self):
        print(f"The value is {self.value}")




In [2]:
import pickle
my_object = MyExampleClass(42)
with open("my_class.pkl", "wb") as file:
    pickle.dump(my_object, file)

In [4]:
with open("my_class.pkl", "rb") as file:
    new_class = pickle.load(file)
new_class.value
    

42